In [2]:
from datasets import load_dataset
import torch

In [3]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"
elif torch.backends.mps.is_available():
	device = torch.device("mps")

In [6]:
# %%capture
# arxiv_abstracts_2021 = load_dataset("gfissore/arxiv-abstracts-2021",split="train")

In [7]:
# arxiv_abstracts_2021[0]

{'id': '0704.0001',
 'submitter': 'Pavel Nadolsky',
 'authors': "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan",
 'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies',
 'comments': '37 pages, 15 figures; published version',
 'journal-ref': 'Phys.Rev.D76:013009,2007',
 'doi': '10.1103/PhysRevD.76.013009',
 'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\nmore detailed tests with C

In [8]:
# %%capture
arxiv_summarization = load_dataset("ccdv/arxiv-summarization",split="train")

In [9]:
arxiv_summarization[0]

{'article': 'additive models @xcite provide an important family of models for semiparametric regression or classification . some reasons for the success of additive models are their increased flexibility when compared to linear or generalized linear models and their increased interpretability when compared to fully nonparametric models . \n it is well - known that good estimators in additive models are in general less prone to the curse of high dimensionality than good estimators in fully nonparametric models . \n many examples of such estimators belong to the large class of regularized kernel based methods over a reproducing kernel hilbert space @xmath0 , see e.g. @xcite . in the last years \n many interesting results on learning rates of regularized kernel based models for additive models have been published when the focus is on sparsity and when the classical least squares loss function is used , see e.g. @xcite , @xcite , @xcite , @xcite , @xcite , @xcite and the references therein

In [32]:
arxiv_summarization = arxiv_summarization.train_test_split(test_size=0.2)

In [33]:
arxiv_summarization["train"][0]

{'article': "clusters of galaxies represent unique signposts in the universe , where the physical properties of the cosmic diffuse baryons can be studied in great details and used to trace the past history of cosmic structure formation ( e.g. rosati et al . \n 2002 ; voit 2005 , for reviews ) . as a result of adiabatic compression and shocks generated by supersonic motion during shell crossing and virialization , \n a hot thin gas permeating the cluster gravitational potential well is formed . \n typically this gas , which is enriched with metals ejected form supernovae ( sne ) explosions through subsequent episodes of star formation ( e.g. matteucci & vettolani 1988 ; renzini 1997 ) , reaches temperatures of several @xmath7 k and therefore emits mainly via thermal bremsstrahlung in the x - rays . at such temperatures most of the elements are either fully ionized or in a high ionization state . \n + particularly evident in x  ray spectra of galaxy clusters are the strong transitions to

In [10]:
from transformers import AutoTokenizer

In [11]:
model_checkpoint = "t5-small"

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [13]:
prefix = "summarize: "

In [18]:
def summarize_preprocess_function(article_data):
    inputs = [prefix + abstract for abstract in article_data["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text_target=article_data["abstract"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [34]:
tokenized_arxiv_summarization = arxiv_summarization.map(summarize_preprocess_function, batched=True)

Map:   0%|          | 0/162429 [00:00<?, ? examples/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 tokenized_arxiv_summarization = arxiv_summarization.map(summarize_preprocess_function, b     │
│   2                                                                                              │
│                                                                                                  │
│ /opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/datasets/dataset_dict.py:851  │
│ in map                                                                                           │
│                                                                                                  │
│    848 │   │   if cache_file_names is None:                                                      │
│    849 │   │   │   cache_file_names = {k: None for k in self}                                    │
│    850 │   │   return DatasetDict(                                                               │
│ ❱  851 │   │   │   {                                                                             │
│    852 │   │   │   │   k: dataset.map(                                                           │
│    853 │   │   │   │   │   function=function,                                                    │
│    854 │   │   │   │   │   with_indices=with_indices,                                            │
│                                                                                                  │
│ /opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/datasets/dataset_dict.py:852  │
│ in <dictcomp>                                                                                    │
│                                                                                                  │
│    849 │   │   │   cache_file_names = {k: None for k in self}                                    │
│    850 │   │   return DatasetDict(                                                               │
│    851 │   │   │   {                                                                             │
│ ❱  852 │   │   │   │   k: dataset.map(                                                           │
│    853 │   │   │   │   │   function=function,                                                    │
│    854 │   │   │   │   │   with_indices=with_indices,                                            │
│    855 │   │   │   │   │   with_rank=with_rank,                                                  │
│                                                                                                  │
│ /opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/datasets/arrow_dataset.py:563 │
│ in wrapper                                                                                       │
│                                                                                                  │
│    560 │   │   else:                                                                             │
│    561 │   │   │   self: "Dataset" = kwargs.pop("self")                                          │
│    562 │   │   # apply actual function                                                           │
│ ❱  563 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)                │
│    564 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) else [ou  │
│    565 │   │   for dataset in datasets:                                                          │
│    566 │   │   │   # Remove task templates if a column mapping of the template is no longer val  │
│                                                                                                  │
│ /opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-p

In [20]:
from transformers import DataCollatorForSeq2Seq

In [21]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_checkpoint)

In [23]:
import evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
rouge = evaluate.load("rouge")

In [26]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [27]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [28]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

/opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/transformers/modeling_utils.py:386: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(checkpoint_file, framework="pt") as f:
/opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/torch/_utils.py:777: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/torch/storage.py:955: UserWarning: TypedStorage is deprecated. It will

In [30]:
training_args = Seq2SeqTrainingArguments(
    output_dir="arxiv_summarization_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
)

In [31]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_arxiv_summarization["train"],
    eval_dataset=tokenized_arxiv_summarization["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│    1 trainer = Seq2SeqTrainer(                                                                   │
│    2 │   model=model,                                                                            │
│    3 │   args=training_args,                                                                     │
│ ❱  4 │   train_dataset=tokenized_arxiv_summarization["train"],                                   │
│    5 │   eval_dataset=tokenized_arxiv_summarization["test"],                                     │
│    6 │   tokenizer=tokenizer,                                                                    │
│    7 │   data_collator=data_collator,                                                            │
│                                                                                                  │
│ /opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/datasets/arrow_dataset.py:270 │
│ 9 in __getitem__                                                                                 │
│                                                                                                  │
│   2706 │                                                                                         │
│   2707 │   def __getitem__(self, key):  # noqa: F811                                             │
│   2708 │   │   """Can be used to index columns (by string names) or rows (by integer index or i  │
│ ❱ 2709 │   │   return self._getitem(key)                                                         │
│   2710 │                                                                                         │
│   2711 │   def __getitems__(self, keys: List) -> List:                                           │
│   2712 │   │   """Can be used to get a batch using a list of integers indices."""                │
│                                                                                                  │
│ /opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/datasets/arrow_dataset.py:269 │
│ 3 in _getitem                                                                                    │
│                                                                                                  │
│   2690 │   │   format_kwargs = kwargs["format_kwargs"] if "format_kwargs" in kwargs else self._  │
│   2691 │   │   format_kwargs = format_kwargs if format_kwargs is not None else {}                │
│   2692 │   │   formatter = get_formatter(format_type, features=self._info.features, **format_kw  │
│ ❱ 2693 │   │   pa_subtable = query_table(self._data, key, indices=self._indices if self._indice  │
│   2694 │   │   formatted_output = format_table(                                                  │
│   2695 │   │   │   pa_subtable, key, formatter=formatter, format_columns=format_columns, output  │
│   2696 │   │   )                                                                                 │
│                                                                                                  │
│ /opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/datasets/formatting/formattin │
│ g.py:585 in query_table                                                                          │
│                                                                                                  │
│   582 │   if not isinstance(key, (int, slice, range, str, Iterable)):                            │
│   583 │   │   _raise_bad_key_type(key)                                                           │
│   584 │   if isinstance(key, str):                                                               │
│ ❱ 585 │   │   _check_valid_column_key(key, table.column_nam